In [ ]:
%matplotlib qt5
#%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams

rcParams.update({
    'font.size': 14, 'grid.alpha': 0.5, 'grid.linestyle': '--',
    'axes.grid': True,
})

# Pymodels

Python module for accelerator modelling

### Accelerator Models

- Linac: li
- Transport Line - Linac to Booster: tb
- Booster: bo
- Transport Line - Booster to Sirius: ts
- Sirius (Storage Ring): si

In [ ]:
import pymodels

In [ ]:
model = pymodels.si.create_accelerator()
model.cavity_on = False
model.radiation_on = False
model.vchamber_on = False

In [ ]:
print(model)

**Family** = A set of objects of exactly same type. 

Examples:
- Central Dipoles (BC)
- Focusing quadrupoles in high beta segments (QFA)
- Beam Position Monitors (BPM)

Getting data about families:

In [ ]:
# fam_name with devname, index, subsection
famdata = pymodels.si.get_family_data(model)

In [ ]:
famdata['BC']['subsection']

In [ ]:
bcidx = famdata['BC']['index']

In [ ]:
bcidx

In [ ]:
np.shape(famdata['BC']['index'])

Some families have devnames keys:

In [ ]:
famdata['BPM']['devnames']

Finding out the families names

In [ ]:
pymodels.si.families.families_pulsed_magnets()

In [ ]:
pymodels.si.families.families_dipoles()

In [ ]:
pymodels.si.families.families_quadrupoles()

In [ ]:
famdata['QFA']['index']

In [ ]:
pymodels.si.families.families_sextupoles()

In [ ]:
# Diagnostic families
pymodels.si.families.families_di()

Extracting PVs information

In [ ]:
from siriuspy.epics import PV

In [ ]:
# Instantiating the first BPM PV whose measures the x position of the beam.
bpm1 = PV(famdata['BPM']['devnames'][3]+':PosY-Mon')

In [ ]:
bpm1.get()

### Elements Types
markers, drifts, dipoles, quadrupoles, sextupoles, cavity

In [ ]:
# Marker
print(model[0])

In [ ]:
# Drift
print(model[2])

In [ ]:
# 20 BC dipoles divided in 34 segments
bc_idx = famdata['BC']['index']
np.array(bc_idx).shape

In [ ]:
bc_elem = model[bc_idx[0][2]]
print(bc_elem) 
# Pay attention at the angle property:

In [ ]:
# quadrupole
q1_idx = famdata['Q1']['index']
np.array(q1_idx).shape

In [ ]:
print(model[q1_idx[0][0]])

In [ ]:
# quadrupole skew
qs_idx = famdata['QS']['index']
np.array(qs_idx).shape

In [ ]:
qs_elem = model[qs_idx[0][0]]
print(qs_elem)

In [ ]:
q1_elem = model[q1_idx[0][0]]
print(q1_elem)

Extracting the KL term of an element

In [ ]:
q1_elem.KL

In [ ]:
q1_elem.polynom_b[1]*q1_elem.length

In [ ]:
# sextupole
sfa0_idx = famdata['SFA0']['index']
np.array(sfa0_idx).shape

In [ ]:
sfa0_elem = model[sfa0_idx[0][0]]
print(sfa0_elem)

Extracting the SL term of an element

In [ ]:
sfa0_elem.SL, sfa0_elem.polynom_b[2]*sfa0_elem.length

In [ ]:
cav_idx = famdata['SRFCav']['index']
cav_elem = model[cav_idx[0][0]]
print(cav_elem)

# Pyaccel

Python module for beam dynamics tracking and optics calculations

accelerator, elements, lattice, optics, tracking

In [ ]:
import pyaccel

### Model manipulation

Getting the $s$ coordinate of all elements in the lattice:

In [ ]:
spos = pyaccel.lattice.find_spos(model)
spos[bc_idx[0]]

Finding indexes of an family present in the model.

In [ ]:
cav_idx = pyaccel.lattice.find_indices(lattice=model, attribute_name='fam_name', value='SRFCav')
cav_idx, famdata['SRFCav']['index']

Getting an attribute

In [ ]:
rf_freq = pyaccel.lattice.get_attribute(model, 'frequency', indices=cav_idx)
rf_freq, model[cav_idx[0]].frequency

Setting an attribute

In [ ]:
pyaccel.lattice.set_attribute(model, 'frequency', indices=cav_idx, values=rf_freq+100)
model[cav_idx[0]].frequency

Setting and getting attributes using python property short command: 

In [ ]:
model[cav_idx[0]].frequency = rf_freq
model[cav_idx[0]].frequency

Change all quadrupoles forces

In [ ]:
# default_KL = model[q1_idx[0][0]].KL

In [ ]:
# pyaccel.lattice.set_attribute(model, 'KL', indices=q1_idx, values=1.5 * default_KL)

### Optics functions

In [ ]:
model = pyaccel.lattice.refine_lattice(model)
twiss, *_ = pyaccel.optics.calc_twiss(model)

## For coupled motion, you can use the Edwards-Teng functions:
#edteng,*_ = pyaccel.optics.calc_edwards_teng(model)  

In [ ]:
betax, betay = twiss.betax, twiss.betay  # Beta functions
etax, etay = twiss.etax, twiss.etay      # Dispersion functions
mux, muy = twiss.mux, twiss.muy          # Phase advance
spos = twiss.spos                        # s coordinate

In [ ]:
plt.figure()
plt.plot(spos, betax, label=r'$\beta_x$')
plt.plot(spos, betay, label=r'$\beta_y$')
plt.xlabel('s [m]')
plt.ylabel(r'$\beta$ [m]')
plt.title('Beta functions')
plt.xlim([0, model.length/5])
plt.ylim([-5, 30])
plt.legend()
pyaccel.graphics.draw_lattice(model, offset=-3, height=3, gca=True)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure()
plt.plot(spos, etax*100, label=r'$\eta_x$', color='C2')
plt.plot(spos, etay*100, label=r'$\eta_y$', color='C3')
plt.xlabel('s [m]')
plt.ylabel(r'$\eta$ [cm]')
plt.title('Dispersion functions')
plt.xlim([0, model.length/20])
plt.ylim([-3, 9])
plt.legend()
pyaccel.graphics.draw_lattice(model, offset=-1.2, height=1.0, gca=True)
plt.tight_layout()
plt.show()

In [ ]:
# Tunes
mux[-1]/2/np.pi, muy[-1]/2/np.pi

In [ ]:
# Chromaticities
pyaccel.optics.get_chromaticities(model)

EdwardsTeng examples

In [ ]:
edteng,*_ = pyaccel.optics.calc_edwards_teng(model)

In [ ]:
C, emit_ratio = pyaccel.optics.estimate_coupling_parameters(edteng)

In [ ]:
C

### Equilibrium Parameters

In [ ]:
# eqparam = pyaccel.optics.EquilibriumParametersIntegrals(model)
eqparam = pyaccel.optics.EqParamsFromBeamEnvelope(model)

In [ ]:
print(eqparam)

In [ ]:
# Momentum Compaction Factor
eqparam.alpha, pyaccel.optics.get_mcf(model)

In [ ]:
# Emittances
eqparam.emit1, eqparam.emit2, eqparam.emit3

In [ ]:
# Energy spread, Bunch length
eqparam.espread0, eqparam.bunlen

In [ ]:
# Damping times
eqparam.tau1, eqparam.tau2, eqparam.tau3

### One-turn matrix

In [ ]:
m1turn = pyaccel.tracking.find_m44(model)

In [ ]:
np.linalg.det(m1turn)

### Closed orbit

$r = [x, x', y, y', \delta, z]^T$

idx = [0, 1, 2, 3, 4, 5]

In [ ]:
model = pymodels.si.create_accelerator()
model.cavity_on = True

In [ ]:
cod = pyaccel.tracking.find_orbit6(model, indices='open')
cod.shape

In [ ]:
plt.figure()
spos = pyaccel.lattice.find_spos(model)
plt.plot(spos, cod[0, :]*1e3, label=r'$x$')
plt.plot(spos, cod[2, :]*1e3, label=r'$y$')
plt.xlabel('s [m]')
plt.ylabel(r'Closed orbit [mm]')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
chidx = famdata['CH']['index'][0][0]
model[chidx].hkick_polynom = 10e-6 # [rad]
codx = pyaccel.tracking.find_orbit6(model, indices='open')
model[chidx].hkick_polynom = 0 # [rad]

cvidx = famdata['CV']['index'][0][0]
model[cvidx].vkick_polynom = 10e-6 # [rad]
cody = pyaccel.tracking.find_orbit6(model, indices='open')
model[cvidx].vkick_polynom = 0 # [rad]

cavidx = famdata['SRFCav']['index'][0][0]
model[cavidx].frequency += 100 # [Hz]
codrf = pyaccel.tracking.find_orbit6(model, indices='open')
model[cavidx].frequency -= 100 # [Hz]

bpmidx = famdata['BPM']['index']
bpmidx = np.array(bpmidx).ravel()

In [ ]:
plt.figure()
spos = pyaccel.lattice.find_spos(model)
plt.plot(spos, codx[0, :]*1e6, label=r'$x$', color='C4')
plt.plot(spos[bpmidx], codx[0, bpmidx]*1e6, '-o', color='C0')
plt.plot(spos, codx[2, :]*1e6, label=r'$y$')
plt.plot(spos[bpmidx], codx[2, bpmidx]*1e6, '-o', color='C1')

plt.xlabel('s [m]')
plt.ylabel(r'Closed orbit [um]')
plt.title('Horizontal Kick')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
plt.figure()
spos = pyaccel.lattice.find_spos(model)
plt.plot(spos, cody[0, :]*1e6, label=r'$x$')
plt.plot(spos[bpmidx], cody[0, bpmidx]*1e6, 'o', color='C0')
plt.plot(spos, cody[2, :]*1e6, label=r'$y$')
plt.plot(spos[bpmidx], cody[2, bpmidx]*1e6, 'o', color='C1')
plt.xlabel('s [m]')
plt.ylabel(r'Closed orbit [um]')
plt.title('Vertical Kick')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
plt.figure()
spos = pyaccel.lattice.find_spos(model)
plt.plot(spos, codrf[0, :]*1e6, label=r'$x$')
plt.plot(spos[bpmidx], codrf[0, bpmidx]*1e6, '.', color='C0')
plt.plot(spos, codrf[2, :]*1e6, label=r'$y$')
plt.plot(spos[bpmidx], codrf[2, bpmidx]*1e6, '.', color='C1')
plt.xlabel('s [m]')
plt.ylabel(r'Closed orbit [um]')
plt.title('RF Frequency Variation')
plt.legend()
plt.tight_layout()
plt.show()

### Tracking

In [ ]:
model = pymodels.si.create_accelerator()
model.cavity_on = False
model.radiation_on = False
model.vchamber_on = True

In [ ]:
x0 = 100e-6
y0 = 1e-6
# nturns = 100

coord_ini = np.array([x0, 0, y0, 0, 0, 0])
coord_fin, *_ = pyaccel.tracking.line_pass(model, coord_ini, indices='open')

In [ ]:
plt.figure()
spos = pyaccel.lattice.find_spos(model)

label = f"$x_0={coord_ini[0]*1e6:.1f}um, x_0'={coord_ini[1]*1e6:.1f}urad$"
plt.plot(spos, coord_fin[0, :]*1e6, ls='-')
plt.xlabel('s [m]')
plt.ylabel("x [um]")
plt.title('Phase Space ' + label)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure()
spos = pyaccel.lattice.find_spos(model)

label = f"$y_0={coord_ini[2]*1e6:.1f}um, y_0'={coord_ini[3]*1e6:.1f}urad$"
plt.plot(spos, coord_fin[2, :]*1e6, ls='-')
plt.xlabel('s [m]')
plt.ylabel("y [um]")
plt.title('Phase Space ' + label)
plt.tight_layout()
plt.show()

In [ ]:
x0 = 1e-6
y0 = 1e-6
nturns = 2000

coord_ini = np.array([x0, 0, y0, 0, 0, 0])
coord_fin, *_ = pyaccel.tracking.ring_pass(model, coord_ini, nr_turns=nturns, turn_by_turn=True, parallel=True)

In [ ]:
plt.figure()

label = f"$x_0={coord_ini[0]*1e6:.1f}um, x_0'={coord_ini[1]*1e6:.1f}urad$"
plt.plot(coord_fin[0, :]*1e6, coord_fin[1, :]*1e6, marker='.', ls='')
plt.xlabel('$x$ [um]')
plt.ylabel(r"$x'$ [urad]")
plt.title('Phase Space ' + label)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure()

label = f"$y_0={coord_ini[2]*1e6:.1f}um, y_0'={coord_ini[3]*1e6:.1f}urad$"
plt.plot(coord_fin[2, :]*1e6, coord_fin[3, :]*1e6, marker='.', ls='')
plt.xlabel('$y$ [um]')
plt.ylabel(r"$y'$ [urad]")
plt.title('Phase Space ' + label)
plt.tight_layout()
plt.show()

In [ ]:
# Switch to 6D tracking
model.cavity_on = True
model.radiation_on = True
orb = pyaccel.tracking.find_orbit6(model, indices=None).ravel()

x0 = 5e-3
nturns = 5000

coord_ini = np.array([x0, 0, 1e-6, 0, 0, 0]) + orb
coord_fin, *_ = pyaccel.tracking.ring_pass(model, coord_ini, nr_turns=nturns, turn_by_turn=True, parallel=True)

In [ ]:
plt.figure()

label = f"$x_0={coord_ini[0]*1e6:.1f}um, x_0'={coord_ini[1]*1e6:.1f}urad$"
plt.plot(coord_fin[0, :]*1e6, coord_fin[1, :]*1e6, marker='.', ls='')
plt.xlabel('$x$ [um]')
plt.ylabel(r"$x'$ [urad]")
plt.title('Phase Space ' + label)
plt.tight_layout()
plt.show()

#### Tracking a bunch of particles

In [ ]:
# 4D first then switch to 6D
model.cavity_on = False
model.radiation_on = False

In [ ]:
emit1 = eqparam.emit1
emit2 = abs(eqparam.emit2)
espread = eqparam.espread0
bunlen = eqparam.bunlen
npart = 200

bunch = pyaccel.tracking.generate_bunch(
    emit1=emit1, emit2=emit2, sigmae=espread, sigmas=bunlen, optics=twiss[0], n_part=npart, cutoff=3)
bunch.shape

In [ ]:
nturns = 500
bunch_tbt, *_ = pyaccel.tracking.ring_pass(model, bunch, nr_turns=nturns, turn_by_turn=True, parallel=True)
bunch_tbt.shape

In [ ]:
plt.figure(figsize=(12, 8))

for part in range(10):
    label = f"$x_0={bunch_tbt[0, part, 0]*1e6:+04.0f}um, x_0'={bunch_tbt[1, part, 0]*1e6:+04.0f}urad$"
    plt.plot(bunch_tbt[0, part, :]*1e6, bunch_tbt[1, part, :]*1e6, marker='.', ls='', label=label)

plt.xlabel('$x$ [um]')
plt.ylabel(r"$x'$ [urad]")
plt.title('Phase Space ')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.tight_layout()
plt.show()

In [ ]:
bunch[0] += 5e-3
nturns = 100
bunch_tbt2, *_ = pyaccel.tracking.ring_pass(model, bunch, nr_turns=nturns, turn_by_turn=True, parallel=True)
bunch_tbt2.shape

In [ ]:
%matplotlib qt5

In [ ]:
plt.figure(figsize=(8, 8))
spos = pyaccel.lattice.find_spos(model)

nt = bunch_tbt2.shape[2]-1
cmap = plt.cm.jet(np.linspace(0, 1, nt+1))
plt.xlim([-7e3, 6e3])
plt.ylim([-400, 400])
plt.xlabel('$x$ [um]')
plt.ylabel(r"$x'$ [urad]")
plt.title('Phase Space ')
plt.tight_layout()
for i, cor in enumerate(cmap):
    plt.plot(bunch_tbt2[0, :, i]*1e6, bunch_tbt2[1, :, i]*1e6, marker='.', ls='', color=cor)
    plt.pause(0.5)
plt.show()

In [ ]:
plt.figure(figsize=(8, 8))
for i in range(npart):
    plt.plot(bunch_tbt2[0, i, :]*1e6, color='C0', alpha=0.01)

plt.plot(np.mean(bunch_tbt2[0, :, :], axis=0)*1e6, lw=2, color='tab:red', label='Centroid')
plt.xlabel('turn index')
plt.ylabel(r"$x$ [um]")
plt.legend()
plt.tight_layout()
plt.show()

### Lifetime

In [ ]:
model = pymodels.si.create_accelerator()
model = pymodels.si.fitted_models.vertical_dispersion_and_coupling(model)
cav = pyaccel.lattice.find_indices(model, 'fam_name', 'SRFCav')[0]
model[cav].voltage = 1.5e6
model.cavity_on=True
model.radiation_on=True
model.vchamber_on=True

In [ ]:
ltime = pyaccel.lifetime.Lifetime(model)
# spos = twiss.spos
spos = pyaccel.lattice.find_spos(model, indices='closed')

In [ ]:
ltime.curr_per_bunch = 100/864
# ltime.avg_pressure = 2e-9

In [ ]:
ltime.lifetime_touschek/3600, float(ltime.lifetime_total/3600)

In [ ]:
enaccn, enaccp = pyaccel.optics.calc_touschek_energy_acceptance(model)
enaccn = np.maximum(enaccn, -ltime.equi_params.rf_acceptance)
enaccp = np.minimum(enaccp, ltime.equi_params.rf_acceptance)
ltime.accepen = (enaccn, enaccp)

Lifetime 

In [ ]:
plt.figure()
plt.plot(spos, enaccn*100)
plt.plot(spos, enaccp*100)
plt.xlabel('s [m]')
plt.ylabel(r'Touschek Energy Acceptance [%]')
plt.title('Energy Acceptance')
plt.xlim([0, model.length/20])
pyaccel.graphics.draw_lattice(model, offset=0, height=1, gca=True)
plt.tight_layout()
plt.show()

In [ ]:
ltime.lifetime_touschek/3600, float(ltime.lifetime_total/3600)